In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import requests
from datetime import datetime

# setting pandas
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# setting seaborn
sns.set_palette('Spectral')
sns.set_context('notebook', font_scale=1)
sns.set_style('whitegrid')
sns.set(rc={"axes.facecolor":"#FFF9ED","figure.facecolor":"#FFF9ED"})

warnings.filterwarnings('ignore')

# ETL

In [2]:
df = pd.read_excel('./Report/task2.xlsx')

In [3]:
# bersihkan data stage kosong dengan stanby
df['stage'].fillna('Stanby', inplace=True)

In [4]:
# transformasi tipe data date
df['date'] = pd.to_datetime(df.date)

df['month'] = df.date.dt.strftime('%m/%y')
df = df[['date','month','project/name','stage','spk_project','spk_workshop','category_equipment',
         'equipment','jam_produktivitas','jam_breakdown','jam_standby','PA','MA','UA','EU']]

In [5]:
df = df.rename(columns={
    'jam_produktivitas':'Actual Working Hours',
    'jam_breakdown':'Total B/D',
    'jam_standby':'Total Standby'
})

# EDA

In [6]:
eda = df[df.month=='03/24']

## Mining BDM

In [7]:
mining_bdm = [
    'PC500-03','PC400-03','PC400-07','PC300-12','PC300-17','PC200-11','PC200-15','PC200-21','PC200-23','PC200-24',
    'HM400-06','HM400-07','HM400-08','HM400-09','HM400-10',
    'D65-11','D65-12','D85-11','SAKAI-01',
    'LV-06','LV-27','LV-28','LV-31','BUS-04','FT-02','LT-06','LT-07'
]

In [8]:
bdm = eda[eda.equipment.isin(mining_bdm)]
bdm = bdm[bdm['project/name']!='Mining KBM']

In [9]:
def get_id(date, project, stage, equipment):
    date = str(date).split()[0]
    id = '#'.join([date, project, stage, equipment])
    return id

In [10]:
bdm['id'] = bdm.apply(lambda x: 
                      get_id(x['date'], x['project/name'], x['stage'], x['equipment'])
                      , axis=1)

In [11]:
bdm.drop_duplicates(subset='id', inplace=True)

In [12]:
bdm.groupby(['month','category_equipment','equipment'])[['Actual Working Hours','Total B/D','Total Standby']].sum()

Actual Working Hours  Total B/D  \
month category_equipment equipment                                    
03/24 HEAVY EQUIPMENT    D65-11                      235   1.184167   
                         D65-12                      190   1.062778   
                         D85-11                      248  10.318611   
                         HM400-06                    251   0.714167   
                         HM400-07                    265   2.211111   
                         HM400-08                    256   5.123333   
                         HM400-09                    225  19.108056   
                         HM400-10                    203  23.440278   
                         PC200-11                    200  21.356389   
                         PC200-15                      0   0.000000   
                         PC200-21                    250  18.050833   
                         PC200-23                    260   8.670278   
                         PC200-24                    270   0.991944   
                         PC300-12                    250   3.188333   
                         PC300-17                    254  19.108611   
                         PC400-03                     20   0.000000   
                         PC400-07                    259   1.481389   
                         PC500-03                    255   1.406111   
                         SAKAI-01                    183   2.215000   
      SUPPORT            BUS-04                        0   0.000000   
                         FT-02                       231  25.068333   
                         LT-06                         0   2.759444   
                         LT-07                        20   0.000000   
                         LV-06                         0  13.595556   
                         LV-27                         0  30.540278   
                         LV-28                         0   6.655278   
                         LV-31                         0   0.620556   

                                    Total Standby  
month category_equipment equipment                 
03/24 HEAVY EQUIPMENT    D65-11         75.000000  
                         D65-12        120.000000  
                         D85-11         52.000000  
                         HM400-06       59.000000  
                         HM400-07       44.667500  
                         HM400-08       49.718611  
                         HM400-09       80.000000  
                         HM400-10       99.345833  
                         PC200-11       99.013333  
                         PC200-15      310.000000  
                         PC200-21       59.833333  
                         PC200-23       50.000000  
                         PC200-24       50.000000  
                         PC300-12       60.000000  
                         PC300-17       56.000000  
                         PC400-03      290.000000  
                         PC400-07       50.325833  
                         PC500-03       54.837500  
                         SAKAI-01      126.668056  
      SUPPORT            BUS-04        310.000000  
                         FT-02          65.034167  
                         LT-06         307.240556  
                         LT-07         240.000000  
                         LV-06         366.404444  
                         LV-27         279.459722  
                         LV-28         303.344722  
                         LV-31         309.379444